In [3]:
import os
import logging
import pandas as pd
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("combined_script.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения к ClickHouse
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def get_initial_date():
    try:
        query = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        result = client.execute(query)
        initial_date = result[0][0]
        logger.info(f"Начальная дата в OlapCube_VNV: {initial_date}")
        return initial_date
    except Exception as e:
        logger.error(f"Ошибка при получении начальной даты: {e}", exc_info=True)
        return None

def process_day(target_date):
    previous_date = target_date - timedelta(days=1)

    # Извлекаем данные за предыдущую дату
    query_prev = f"""
    SELECT 
        serialno, 
        Status AS Status_prev, 
        Status_P AS Status_P_prev,
        sne AS sne_prev, 
        ppr AS ppr_prev, 
        repair_days AS repair_days_prev
    FROM {database_name}.OlapCube_VNV
    WHERE Dates = '{previous_date}'
    """
    prev_data = client.execute(query_prev)
    prev_df = pd.DataFrame(prev_data, columns=[
        'serialno','Status_prev','Status_P_prev','sne_prev','ppr_prev','repair_days_prev'
    ])

    # Извлекаем данные за target_date
    query_target = f"""
    SELECT 
        serialno, Status, Status_P, sne, ppr, repair_days, ll, oh, BR, daily_flight_hours, RepairTime, ac_typ, mi8t_count, mi17_count
    FROM {database_name}.OlapCube_VNV
    WHERE Dates = '{target_date}'
    """
    target_data = client.execute(query_target)
    target_df = pd.DataFrame(target_data, columns=[
        'serialno','Status','Status_P','sne','ppr','repair_days','ll','oh','BR','daily_flight_hours','RepairTime','ac_typ','mi8t_count','mi17_count'
    ])

    # Объединяем данные
    intermediate_table = pd.merge(target_df, prev_df, on='serialno', how='left')

    # -----------------------
    # Применяем логику вычислений (из вашего фрагмента)
    # -----------------------

    # Инициализация Status_P
    intermediate_table['Status_P'] = None

    # Копирование статусов "Неактивно", "Хранение", "Исправен"
    intermediate_table.loc[intermediate_table['Status_prev'] == 'Неактивно', 'Status_P'] = 'Неактивно'
    intermediate_table.loc[intermediate_table['Status_prev'] == 'Хранение', 'Status_P'] = 'Хранение'
    intermediate_table.loc[intermediate_table['Status_prev'] == 'Исправен', 'Status_P'] = 'Исправен'

    # Логика для "Ремонт"
    intermediate_table.loc[
        (intermediate_table['Status_prev'] == 'Ремонт') & 
        (intermediate_table['repair_days_prev'] < intermediate_table['RepairTime']),
        'Status_P'
    ] = 'Ремонт'
    intermediate_table.loc[
        (intermediate_table['Status_prev'] == 'Ремонт') & 
        (intermediate_table['repair_days_prev'] >= intermediate_table['RepairTime']),
        'Status_P'
    ] = 'Исправен'

    # Логика для "Эксплуатация"
    intermediate_table.loc[
        (intermediate_table['Status_prev'] == 'Эксплуатация') &
        (intermediate_table['sne_prev'] < (intermediate_table['ll'] - intermediate_table['daily_flight_hours'])) &
        (intermediate_table['ppr_prev'] < (intermediate_table['oh'] - intermediate_table['daily_flight_hours'])),
        'Status_P'
    ] = 'Эксплуатация'

    intermediate_table.loc[
        (intermediate_table['Status_prev'] == 'Эксплуатация') &
        (intermediate_table['sne_prev'] >= (intermediate_table['ll'] - intermediate_table['daily_flight_hours'])),
        'Status_P'
    ] = 'Хранение'

    def exploitation_to_repair_or_storage(row):
        if (row['Status_prev'] == 'Эксплуатация') and (row['ppr_prev'] >= (row['oh'] - row['daily_flight_hours'])):
            return 'Ремонт' if row['sne_prev'] < row['BR'] else 'Хранение'
        return row['Status_P']

    intermediate_table['Status_P'] = intermediate_table.apply(exploitation_to_repair_or_storage, axis=1)

    # Вычисление балансов и запасов
    if len(intermediate_table) > 0:
        mi8t_count = intermediate_table['mi8t_count'].iloc[0]
        mi17_count = intermediate_table['mi17_count'].iloc[0]
    else:
        mi8t_count = 0
        mi17_count = 0

    balance_mi8t_count = len(intermediate_table[(intermediate_table['Status_P']=='Эксплуатация') & (intermediate_table['ac_typ']=='Ми-8Т')])
    intermediate_table['balance_mi8t'] = balance_mi8t_count - mi8t_count

    balance_mi17_count = len(intermediate_table[(intermediate_table['Status_P']=='Эксплуатация') & (intermediate_table['ac_typ']=='Ми-17')])
    intermediate_table['balance_mi17'] = balance_mi17_count - mi17_count

    balance_empty_count = len(intermediate_table[(intermediate_table['Status_P']=='Эксплуатация') & (intermediate_table['ac_typ'].isna())])
    intermediate_table['balance_empty'] = balance_empty_count

    intermediate_table['balance_total'] = intermediate_table['balance_mi8t'] + intermediate_table['balance_mi17'] + intermediate_table['balance_empty']

    stock_mi8t = len(intermediate_table[(intermediate_table['Status_P']=='Исправен')&(intermediate_table['ac_typ']=='Ми-8Т')])
    intermediate_table['stock_mi8t'] = stock_mi8t

    stock_mi17 = len(intermediate_table[(intermediate_table['Status_P']=='Исправен')&(intermediate_table['ac_typ']=='Ми-17')])
    intermediate_table['stock_mi17'] = stock_mi17

    stock_empty = len(intermediate_table[(intermediate_table['Status_P']=='Исправен') & (intermediate_table['ac_typ'].isna())])
    intermediate_table['stock_empty'] = stock_empty

    intermediate_table['stock_total'] = intermediate_table['stock_mi8t'] + intermediate_table['stock_mi17'] + intermediate_table['stock_empty']

    # Балансировка статусов
    balance_total = intermediate_table['balance_total'].iloc[0] if len(intermediate_table) > 0 else 0
    if balance_total > 0:
        exploita_indexes = intermediate_table[(intermediate_table['Status_P']=='Эксплуатация')].index
        to_fix = exploita_indexes[:balance_total]
        intermediate_table.loc[to_fix,'Status_P']='Исправен'
    elif balance_total < 0:
        toadd = abs(balance_total)
        isprav_indexes = intermediate_table[(intermediate_table['Status_P']=='Исправен')].index
        to_exploit = isprav_indexes[:toadd]
        intermediate_table.loc[to_exploit,'Status_P']='Эксплуатация'
        remain = toadd - len(to_exploit)
        if remain>0:
            neaktiv_indexes = intermediate_table[(intermediate_table['Status_P']=='Неактивно')].index
            to_exploit_inactive = neaktiv_indexes[:remain]
            intermediate_table.loc[to_exploit_inactive,'Status_P']='Эксплуатация'

    # Обновление счетчиков sne, ppr, repair_days
    is_explo = (intermediate_table['Status_P']=='Эксплуатация')
    intermediate_table.loc[is_explo, 'sne'] = intermediate_table.loc[is_explo,'sne_prev'] + intermediate_table.loc[is_explo,'daily_flight_hours']
    intermediate_table.loc[is_explo, 'ppr'] = intermediate_table.loc[is_explo,'ppr_prev'] + intermediate_table.loc[is_explo,'daily_flight_hours']

    is_ispr = (intermediate_table['Status_P']=='Исправен')
    intermediate_table.loc[is_ispr,'sne'] = intermediate_table.loc[is_ispr,'sne_prev']
    was_repair = (intermediate_table['Status_P_prev']=='Ремонт')
    intermediate_table.loc[is_ispr & was_repair,'ppr'] = 0
    intermediate_table.loc[is_ispr & (~was_repair),'ppr'] = intermediate_table.loc[is_ispr & (~was_repair),'ppr_prev']

    is_repair = (intermediate_table['Status_P']=='Ремонт')
    intermediate_table.loc[is_repair,'sne'] = intermediate_table.loc[is_repair,'sne_prev']
    intermediate_table.loc[is_repair,'ppr'] = intermediate_table.loc[is_repair,'ppr_prev']
    was_exploit = (intermediate_table['Status_P_prev']=='Эксплуатация')
    intermediate_table.loc[is_repair & was_exploit,'repair_days'] = 1
    intermediate_table.loc[is_repair & (~was_exploit),'repair_days'] = intermediate_table.loc[is_repair & (~was_exploit),'repair_days_prev'] + 1

    store_inactive = intermediate_table['Status_P'].isin(['Хранение','Неактивно'])
    intermediate_table.loc[store_inactive,'sne'] = intermediate_table.loc[store_inactive,'sne_prev']
    intermediate_table.loc[store_inactive,'ppr'] = intermediate_table.loc[store_inactive,'ppr_prev']
    intermediate_table.loc[store_inactive,'repair_days'] = None

    # Обновление данных в базе
    # Предполагается, что все новые значения Status_P, sne, ppr, repair_days, балансы и запасы
    # нужно обновить для каждой строки.
    for idx, row in intermediate_table.iterrows():
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            Status_P = %(Status_P)s,
            sne = %(sne)s,
            ppr = %(ppr)s,
            repair_days = %(repair_days)s,
            balance_mi8t = %(balance_mi8t)s,
            balance_mi17 = %(balance_mi17)s,
            balance_total = %(balance_total)s,
            stock_mi8t = %(stock_mi8t)s,
            stock_mi17 = %(stock_mi17)s,
            stock_total = %(stock_total)s
        WHERE serialno = %(serialno)s AND Dates = %(date)s
        """
        params = {
            'Status_P': row['Status_P'],
            'sne': row['sne'],
            'ppr': row['ppr'],
            'repair_days': row['repair_days'],
            'balance_mi8t': row['balance_mi8t'],
            'balance_mi17': row['balance_mi17'],
            'balance_total': row['balance_total'],
            'stock_mi8t': row['stock_mi8t'],
            'stock_mi17': row['stock_mi17'],
            'stock_total': row['stock_total'],
            'serialno': row['serialno'],
            'date': target_date
        }
        try:
            client.execute(update_query, params)
        except Exception as e:
            logger.error(f"Ошибка обновления для serialno {row['serialno']} и даты {target_date}: {e}", exc_info=True)

    logger.info(f"Обработка даты {target_date} завершена.")

def main(days_to_process=1):
    initial_date = get_initial_date()
    if not initial_date:
        logger.error("Не удалось определить начальную дату.")
        return

    # Приводим initial_date к дате без времени
    if isinstance(initial_date, datetime):
        initial_date = initial_date.date()

    for i in range(days_to_process):
        target_date = initial_date + timedelta(days=i+1)
        process_day(target_date)

if __name__ == "__main__":
    # Можно менять количество дней, например 10
    main(days_to_process=2)


2024-12-04 12:50:53,263 - INFO - Начальная дата в OlapCube_VNV: 2024-11-25
2024-12-04 12:51:00,235 - INFO - Обработка даты 2024-11-26 завершена.


TypeError: slice indices must be integers or None or have an __index__ method

In [4]:
import os
import logging
import pandas as pd
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("combined_script.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения к ClickHouse
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def get_initial_date():
    try:
        query = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        result = client.execute(query)
        initial_date = result[0][0]
        logger.info(f"Начальная дата в OlapCube_VNV: {initial_date}")
        return initial_date
    except Exception as e:
        logger.error(f"Ошибка при получении начальной даты: {e}", exc_info=True)
        return None

def process_day(target_date):
    previous_date = target_date - timedelta(days=1)

    # Извлекаем данные за предыдущую дату
    query_prev = f"""
    SELECT 
        serialno, 
        Status AS Status_prev, 
        Status_P AS Status_P_prev,
        sne AS sne_prev, 
        ppr AS ppr_prev, 
        repair_days AS repair_days_prev
    FROM {database_name}.OlapCube_VNV
    WHERE Dates = '{previous_date}'
    """
    prev_data = client.execute(query_prev)
    prev_df = pd.DataFrame(prev_data, columns=[
        'serialno','Status_prev','Status_P_prev','sne_prev','ppr_prev','repair_days_prev'
    ])

    # Извлекаем данные за target_date
    query_target = f"""
    SELECT 
        serialno, Status, Status_P, sne, ppr, repair_days, ll, oh, BR, daily_flight_hours, RepairTime, ac_typ, mi8t_count, mi17_count
    FROM {database_name}.OlapCube_VNV
    WHERE Dates = '{target_date}'
    """
    target_data = client.execute(query_target)
    target_df = pd.DataFrame(target_data, columns=[
        'serialno','Status','Status_P','sne','ppr','repair_days','ll','oh','BR','daily_flight_hours','RepairTime','ac_typ','mi8t_count','mi17_count'
    ])

    # Объединяем данные
    intermediate_table = pd.merge(target_df, prev_df, on='serialno', how='left')

    # -----------------------
    # Применяем логику вычислений
    # -----------------------

    # Инициализация Status_P
    intermediate_table['Status_P'] = None

    # Статусы копируем напрямую
    intermediate_table.loc[intermediate_table['Status_prev'] == 'Неактивно', 'Status_P'] = 'Неактивно'
    intermediate_table.loc[intermediate_table['Status_prev'] == 'Хранение', 'Status_P'] = 'Хранение'
    intermediate_table.loc[intermediate_table['Status_prev'] == 'Исправен', 'Status_P'] = 'Исправен'

    # Логика для "Ремонт"
    intermediate_table.loc[
        (intermediate_table['Status_prev'] == 'Ремонт') & 
        (intermediate_table['repair_days_prev'] < intermediate_table['RepairTime']),
        'Status_P'
    ] = 'Ремонт'
    intermediate_table.loc[
        (intermediate_table['Status_prev'] == 'Ремонт') & 
        (intermediate_table['repair_days_prev'] >= intermediate_table['RepairTime']),
        'Status_P'
    ] = 'Исправен'

    # Логика для "Эксплуатация"
    intermediate_table.loc[
        (intermediate_table['Status_prev'] == 'Эксплуатация') &
        (intermediate_table['sne_prev'] < (intermediate_table['ll'] - intermediate_table['daily_flight_hours'])) &
        (intermediate_table['ppr_prev'] < (intermediate_table['oh'] - intermediate_table['daily_flight_hours'])),
        'Status_P'
    ] = 'Эксплуатация'

    # Эксплуатация -> Хранение
    intermediate_table.loc[
        (intermediate_table['Status_prev'] == 'Эксплуатация') &
        (intermediate_table['sne_prev'] >= (intermediate_table['ll'] - intermediate_table['daily_flight_hours'])),
        'Status_P'
    ] = 'Хранение'

    def exploitation_to_repair_or_storage(row):
        if (row['Status_prev'] == 'Эксплуатация') and (row['ppr_prev'] >= (row['oh'] - row['daily_flight_hours'])):
            return 'Ремонт' if row['sne_prev'] < row['BR'] else 'Хранение'
        return row['Status_P']

    intermediate_table['Status_P'] = intermediate_table.apply(exploitation_to_repair_or_storage, axis=1)

    # Вычисление балансов и запасов
    if len(intermediate_table) > 0:
        mi8t_count = intermediate_table['mi8t_count'].iloc[0] if pd.notnull(intermediate_table['mi8t_count'].iloc[0]) else 0
        mi17_count = intermediate_table['mi17_count'].iloc[0] if pd.notnull(intermediate_table['mi17_count'].iloc[0]) else 0
    else:
        mi8t_count = 0
        mi17_count = 0

    if pd.isna(mi8t_count):
        mi8t_count = 0
    if pd.isna(mi17_count):
        mi17_count = 0

    mi8t_count = int(mi8t_count)
    mi17_count = int(mi17_count)

    balance_mi8t_count = len(intermediate_table[(intermediate_table['Status_P']=='Эксплуатация') & (intermediate_table['ac_typ']=='Ми-8Т')])
    intermediate_table['balance_mi8t'] = balance_mi8t_count - mi8t_count

    balance_mi17_count = len(intermediate_table[(intermediate_table['Status_P']=='Эксплуатация') & (intermediate_table['ac_typ']=='Ми-17')])
    intermediate_table['balance_mi17'] = balance_mi17_count - mi17_count

    balance_empty_count = len(intermediate_table[(intermediate_table['Status_P']=='Эксплуатация') & (intermediate_table['ac_typ'].isna())])
    intermediate_table['balance_empty'] = balance_empty_count

    intermediate_table['balance_total'] = intermediate_table['balance_mi8t'] + intermediate_table['balance_mi17'] + intermediate_table['balance_empty']

    stock_mi8t = len(intermediate_table[(intermediate_table['Status_P']=='Исправен')&(intermediate_table['ac_typ']=='Ми-8Т')])
    intermediate_table['stock_mi8t'] = stock_mi8t

    stock_mi17 = len(intermediate_table[(intermediate_table['Status_P']=='Исправен')&(intermediate_table['ac_typ']=='Ми-17')])
    intermediate_table['stock_mi17'] = stock_mi17

    stock_empty = len(intermediate_table[(intermediate_table['Status_P']=='Исправен') & (intermediate_table['ac_typ'].isna())])
    intermediate_table['stock_empty'] = stock_empty

    intermediate_table['stock_total'] = intermediate_table['stock_mi8t'] + intermediate_table['stock_mi17'] + intermediate_table['stock_empty']

    # Балансировка статусов
    balance_total = intermediate_table['balance_total'].iloc[0] if len(intermediate_table) > 0 else 0
    if pd.isna(balance_total):
        balance_total = 0
    balance_total = int(balance_total)

    if balance_total > 0:
        exploita_indexes = intermediate_table[(intermediate_table['Status_P']=='Эксплуатация')].index
        to_fix = exploita_indexes[:balance_total]  # здесь balance_total целочисленный
        intermediate_table.loc[to_fix,'Status_P']='Исправен'
    elif balance_total < 0:
        toadd = abs(balance_total)
        isprav_indexes = intermediate_table[(intermediate_table['Status_P']=='Исправен')].index
        to_exploit = isprav_indexes[:toadd]
        intermediate_table.loc[to_exploit,'Status_P']='Эксплуатация'
        remain = toadd - len(to_exploit)
        if remain>0:
            neaktiv_indexes = intermediate_table[(intermediate_table['Status_P']=='Неактивно')].index
            to_exploit_inactive = neaktiv_indexes[:remain]
            intermediate_table.loc[to_exploit_inactive,'Status_P']='Эксплуатация'

    # Обновление счетчиков sne, ppr, repair_days

    # Признаки статусов
    is_explo = (intermediate_table['Status_P']=='Эксплуатация')
    is_ispr = (intermediate_table['Status_P']=='Исправен')
    is_repair = (intermediate_table['Status_P']=='Ремонт')
    store_inactive = intermediate_table['Status_P'].isin(['Хранение','Неактивно'])
    was_repair = (intermediate_table['Status_P_prev']=='Ремонт')
    was_exploit = (intermediate_table['Status_P_prev']=='Эксплуатация')

    # Эксплуатация
    intermediate_table.loc[is_explo, 'sne'] = intermediate_table.loc[is_explo,'sne_prev'] + intermediate_table.loc[is_explo,'daily_flight_hours']
    intermediate_table.loc[is_explo, 'ppr'] = intermediate_table.loc[is_explo,'ppr_prev'] + intermediate_table.loc[is_explo,'daily_flight_hours']

    # Исправен
    intermediate_table.loc[is_ispr,'sne'] = intermediate_table.loc[is_ispr,'sne_prev']
    intermediate_table.loc[is_ispr & was_repair,'ppr'] = 0
    intermediate_table.loc[is_ispr & (~was_repair),'ppr'] = intermediate_table.loc[is_ispr & (~was_repair),'ppr_prev']

    # Ремонт
    intermediate_table.loc[is_repair,'sne'] = intermediate_table.loc[is_repair,'sne_prev']
    intermediate_table.loc[is_repair,'ppr'] = intermediate_table.loc[is_repair,'ppr_prev']
    intermediate_table.loc[is_repair & was_exploit,'repair_days'] = 1
    intermediate_table.loc[is_repair & (~was_exploit),'repair_days'] = intermediate_table.loc[is_repair & (~was_exploit),'repair_days_prev'] + 1

    # Хранение или Неактивно
    intermediate_table.loc[store_inactive,'sne'] = intermediate_table.loc[store_inactive,'sne_prev']
    intermediate_table.loc[store_inactive,'ppr'] = intermediate_table.loc[store_inactive,'ppr_prev']
    intermediate_table.loc[store_inactive,'repair_days'] = None

    # Обновление данных в базе
    for idx, row in intermediate_table.iterrows():
        update_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV 
        UPDATE 
            Status_P = %(Status_P)s,
            sne = %(sne)s,
            ppr = %(ppr)s,
            repair_days = %(repair_days)s,
            balance_mi8t = %(balance_mi8t)s,
            balance_mi17 = %(balance_mi17)s,
            balance_total = %(balance_total)s,
            stock_mi8t = %(stock_mi8t)s,
            stock_mi17 = %(stock_mi17)s,
            stock_total = %(stock_total)s
        WHERE serialno = %(serialno)s AND Dates = %(date)s
        """

        params = {
            'Status_P': row['Status_P'],
            'sne': row['sne'],
            'ppr': row['ppr'],
            'repair_days': row['repair_days'],
            'balance_mi8t': row['balance_mi8t'],
            'balance_mi17': row['balance_mi17'],
            'balance_total': row['balance_total'],
            'stock_mi8t': row['stock_mi8t'],
            'stock_mi17': row['stock_mi17'],
            'stock_total': row['stock_total'],
            'serialno': row['serialno'],
            'date': target_date
        }

        try:
            client.execute(update_query, params)
        except Exception as e:
            logger.error(f"Ошибка обновления для serialno {row['serialno']} и даты {target_date}: {e}", exc_info=True)

    logger.info(f"Обработка даты {target_date} завершена.")

def main(days_to_process=2):
    initial_date = get_initial_date()
    if not initial_date:
        logger.error("Не удалось определить начальную дату.")
        return

    # Приводим initial_date к дате без времени
    if isinstance(initial_date, datetime):
        initial_date = initial_date.date()

    for i in range(days_to_process):
        target_date = initial_date + timedelta(days=i+1)
        process_day(target_date)

if __name__ == "__main__":
    # Можно менять количество дней
    main(days_to_process=2)


2024-12-04 13:12:43,928 - INFO - Начальная дата в OlapCube_VNV: 2024-11-25
2024-12-04 13:12:51,089 - INFO - Обработка даты 2024-11-26 завершена.
2024-12-04 13:13:17,274 - INFO - Обработка даты 2024-11-27 завершена.
